# FDS Challenge: Starter Notebook

This notebook will guide you through the first steps of the competition. Our goal here is to show you how to:

1.  Load the `train.jsonl` and `test.jsonl` files from the competition data.
2.  Create a very simple set of features from the data.
3.  Train a basic model.
4.  Generate a `submission.csv` file in the correct format.
5.  Submit your results.

Let's get started!

### 1. Load the train.jsonl and test.jsonl Files from the Competition Data

In [ ]:
# --- Added to mount drive ---
from google.colab import drive
drive.mount('/content/drive')

import json
import pandas as pd
import os

# --- Define the path to our data ---
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.path.join('/content/drive/MyDrive', COMPETITION_NAME)
train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')

# Read the file line by line
train_data = []
print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r') as f:
        for line in f:
            # json.loads() parses one line (one JSON object) into a Python dictionary
            train_data.append(json.loads(line))
    print(f"Successfully loaded {len(train_data)} battles.")

    # Let's inspect the first battle to see its structure
    print("\n--- Structure of the first train battle: ---")
    if train_data:
        first_battle = train_data[0]

        # To keep the output clean, we can create a copy and truncate the timeline
        battle_for_display = first_battle.copy()
        battle_for_display['battle_timeline'] = battle_for_display.get('battle_timeline', [])[-2:] # Show first 2 turns

        # Use json.dumps for pretty-printing the dictionary
        print(json.dumps(battle_for_display, indent=4))
        if len(first_battle.get('battle_timeline', [])) > 3:
            print("    ...")
            print("    (battle_timeline has been truncated for display)")


except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

# Drop the batte 4877 which is wrong
train_data = [battle for battle in train_data if battle.get("battle_id") != 4877]

### 1.5 Supporting Functions and Structures

In [ ]:
def avg_effectiveness_moves(battle):
  """
  Add to the features the average effectivness of the moves of the surviving p1 pokemons relative to p2 lead (if it is still conscious at the end of timeline)
  """

  type_chart = {
    "Normal": {
        "Rock": 0.5, "Steel": 0.5, "Ghost": 0,
    },
    "Fire": {
        "Fire": 0.5, "Water": 0.5, "Grass": 2, "Ice": 2,
        "Bug": 2, "Rock": 0.5, "Dragon": 0.5, "Steel": 2,
    },
    "Water": {
        "Fire": 2, "Water": 0.5, "Grass": 0.5, "Rock": 2, "Ground": 2, "Dragon": 0.5,
    },
    "Grass": {
        "Fire": 0.5, "Water": 2, "Grass": 0.5, "Poison": 0.5,
        "Ground": 2, "Flying": 0.5, "Bug": 0.5, "Rock": 2,
        "Dragon": 0.5, "Steel": 0.5,
    },
    "Electric": {
        "Water": 2, "Grass": 0.5, "Electric": 0.5, "Ground": 0,
        "Flying": 2, "Dragon": 0.5,
    },
    "Ice": {
        "Fire": 0.5, "Water": 0.5, "Grass": 2, "Ice": 0.5,
        "Ground": 2, "Flying": 2, "Dragon": 2, "Steel": 0.5,
    },
    "Fighting": {
        "Normal": 2, "Ice": 2, "Rock": 2, "Dark": 2, "Steel": 2,
        "Poison": 0.5, "Flying": 0.5, "Psychic": 0.5, "Bug": 0.5,
        "Ghost": 0, "Fairy": 0.5,
    },
    "Poison": {
        "Grass": 2, "Poison": 0.5, "Ground": 0.5, "Rock": 0.5,
        "Ghost": 0.5, "Steel": 0, "Fairy": 2,
    },
    "Ground": {
        "Fire": 2, "Grass": 0.5, "Electric": 2, "Poison": 2, "Rock": 2,
        "Bug": 0.5, "Flying": 0, "Steel": 2,
    },
    "Flying": {
        "Grass": 2, "Fighting": 2, "Bug": 2, "Rock": 0.5,
        "Electric": 0.5, "Steel": 0.5,
    },
    "Psychic": {
        "Fighting": 2, "Poison": 2, "Steel": 0.5, "Psychic": 0.5, "Dark": 0,
    },
    "Bug": {
        "Grass": 2, "Psychic": 2, "Dark": 2, "Fire": 0.5, "Fighting": 0.5,
        "Poison": 0.5, "Flying": 0.5, "Ghost": 0.5, "Steel": 0.5,
        "Fairy": 0.5,
    },
    "Rock": {
        "Fire": 2, "Ice": 2, "Flying": 2, "Bug": 2,
        "Fighting": 0.5, "Ground": 0.5, "Steel": 0.5,
    },
    "Ghost": {
        "Normal": 0, "Psychic": 2, "Ghost": 2, "Dark": 0.5,
    },
    "Dragon": {
        "Dragon": 2, "Steel": 0.5, "Fairy": 0,
    },
    "Dark": {
        "Psychic": 2, "Ghost": 2, "Fighting": 0.5, "Dark": 0.5, "Fairy": 0.5,
    },
    "Steel": {
        "Fire": 0.5, "Water": 0.5, "Electric": 0.5, "Ice": 2,
        "Rock": 2, "Fairy": 2, "Steel": 0.5,
    },
    "Fairy": {
        "Fighting": 2, "Dragon": 2, "Dark": 2, "Fire": 0.5,
        "Poison": 0.5, "Steel": 0.5,
    },
}

  p1_team_details = {pokemon.get('name', f'p1_unknown_{i}'):
    {
      'hp': 1.00,
      'types': pokemon.get('types'),
      'moves': []
    } for i, pokemon in enumerate(battle.get('p1_team_details', []))}

  p2_lead_details = {
      'name' : battle.get('p2_lead_details', {}).get('name'),
      'hp' : 1.00,
      'types' : battle.get('p2_lead_details', {}).get('types')
  }

  for turn in battle.get('battle_timeline', []):

    p1_state = turn.get('p1_pokemon_state', {})
    p1_name = p1_state.get('name')
    if p1_name in p1_team_details:
        if 'hp_pct' in p1_state:
            p1_team_details[p1_name]['hp'] = p1_state['hp_pct']

    move_details = turn.get('p1_move_details')
    if move_details:
        move_name = move_details.get('name')
        already_known = any(m.get('name') == move_name for m in p1_team_details[p1_name]['moves'])
        if not already_known:
            p1_team_details[p1_name]['moves'].append(move_details)

    p2_state = turn.get('p2_pokemon_state', {})
    if p2_state.get('name') == p2_lead_details['name']:
      p2_lead_details['hp'] = p2_state['hp_pct']

  # Check of the lead pokemon
  if p2_lead_details['hp'] == 0:
    return 0
  else:
    total_effectiveness = 0
    move_count = 0

    # Types of the lead of p2
    p2_types = p2_lead_details.get('types', [])

    for p1_name, p1_data in p1_team_details.items():
      # Only not fainted pokemons
      if p1_data['hp'] > 0:
        for move in p1_data['moves']:
          if move['base_power'] == 0:
            continue
          else:
            move_type = move.get('type')
            multiplier = 1.0
            for target_type in p2_types:
                multiplier *= type_chart.get(move_type, {}).get(target_type, 1.0)

            total_effectiveness += multiplier
            move_count += 1

    if move_count == 0:
        return 0

    avg_effectiveness = total_effectiveness / move_count
    return avg_effectiveness

def create_pokemon_stats_dict(data_list: list[dict]):
    """
    Iterates through a list of battle data and extracts the base stats and types
    for every unique Pokémon, returning a dictionary mapping name to stats and types
    """
    pokemon_stats = {}

    for battle in data_list:
        # Check P1 team details
        pokemon_list = battle.get('p1_team_details', [])
        for pokemon in pokemon_list:
            name = pokemon.get('name')
            lvl = pokemon.get('level')
            if name and name not in pokemon_stats and lvl==100:
                # Extract all base stats
                stats = {k: v for k, v in pokemon.items() if k.startswith('base_')}

                # Add the types
                stats['types'] = pokemon.get('types', ['notype', 'notype'])

                if stats:
                    pokemon_stats[name] = stats
    return pokemon_stats

def track_pokemon_conditions(battle):
    """
    Iterates through the timeline of a battle to track the conditions of each Pokémon
    """
    # Initialize the data structure
    p1_pok_cond = {pokemon.get('name', f'p1_unknown_{i}'):
    {
      'hp': 1.00,
      'status': 'nostatus'
    } for i, pokemon in enumerate(battle.get('p1_team_details', []))}
    p2_pok_cond = {}
    p2_pok_cond[battle.get('p2_lead_details', {}).get('name')] = {
      'hp': 1.00,
      'status': 'nostatus'
    }

    # Fill the values with the latest conditions shown in the timeline
    for turn in battle.get('battle_timeline', []):
      p1_pok_cond[turn.get('p1_pokemon_state', {}).get('name')] = {
          'hp': turn.get('p1_pokemon_state', {}).get('hp_pct'),
          'status': turn.get('p1_pokemon_state', {}).get('status')
      }
      p2_pok_cond[turn.get('p2_pokemon_state', {}).get('name')] = {
          'hp': turn.get('p2_pokemon_state', {}).get('hp_pct'),
          'status': turn.get('p2_pokemon_state', {}).get('status')
      }

    # Compute the number of pokemon changes for each player (indicator of strategy)
    p1_n_changes = 0
    p2_n_changes = 0
    for turn in battle.get('battle_timeline', []):
      p1_current_pok = turn.get('p1_pokemon_state', {}).get('name')
      p2_current_pok = turn.get('p2_pokemon_state', {}).get('name')
      if turn != battle.get('battle_timeline', [])[0]:
        if p1_pre_pok != p1_current_pok:
          p1_n_changes += 1
        if p2_pre_pok != p2_current_pok:
          p2_n_changes += 1
      else:
        p1_pre_pok = p1_current_pok
        p2_pre_pok = p2_current_pok

    # Compute the number of effects working on the last round and weigh them
    p1_effects = len(battle.get('battle_timeline', [])[-1].get('p1_pokemon_state', {}).get('effects', []))*0.4
    p2_effects = len(battle.get('battle_timeline', [])[-1].get('p2_pokemon_state', {}).get('effects', []))*0.4

    # Add the slots corresponding to the unseen pokemons of player #2
    for i in range(len(p2_pok_cond), 6):
      p2_pok_cond[f'p2_unknown_{i}'] = {
          'hp': 1.00,
          'status': 'nostatus'
      }
    return p1_n_changes, p1_effects, p1_pok_cond, p2_n_changes, p2_effects, p2_pok_cond

def compute_differences_base_stats(p1_pok_cond, p2_pok_cond, pokemon_dict):
  """
  Calculates the difference in total base stats between player #1 and player #2
  """
  p1_total_speed = 0
  p2_total_speed = 0
  p1_total_attack = 0
  p2_total_attack = 0
  p1_total_defense = 0
  p2_total_defense = 0
  p1_total_sp_attack = 0
  p2_total_sp_attack = 0
  p1_total_sp_defense = 0
  p2_total_sp_defense = 0
  p1_total_hp = 0
  p2_total_hp = 0
  for pokemon in p1_pok_cond.keys():
    if pokemon in pokemon_dict:
      p1_total_speed += pokemon_dict[pokemon]['base_spe']
      p1_total_attack += pokemon_dict[pokemon]['base_atk']
      p1_total_defense += pokemon_dict[pokemon]['base_def']
      p1_total_sp_attack += pokemon_dict[pokemon]['base_spa']
      p1_total_sp_defense += pokemon_dict[pokemon]['base_spd']
      p1_total_hp += pokemon_dict[pokemon]['base_hp']
  for pokemon in p2_pok_cond.keys():
    if pokemon in pokemon_dict:
      p2_total_speed += pokemon_dict[pokemon]['base_spe']
      p2_total_attack += pokemon_dict[pokemon]['base_atk']
      p2_total_defense += pokemon_dict[pokemon]['base_def']
      p2_total_sp_attack += pokemon_dict[pokemon]['base_spa']
      p2_total_sp_defense += pokemon_dict[pokemon]['base_spd']
      p2_total_hp += pokemon_dict[pokemon]['base_hp']
  speed = p1_total_speed-p2_total_speed
  defense = p1_total_defense-p2_total_defense
  attack = p1_total_attack-p2_total_attack
  sp_attack = p1_total_sp_attack-p2_total_sp_attack
  sp_defense = p1_total_sp_defense-p2_total_sp_defense
  hp = p1_total_hp-p2_total_hp
  return speed, defense, attack, sp_attack, sp_defense, hp

# TO BE REMOVED
def get_all_statuses(data: list[dict]):
    all_statuses = set()

    for battle in tqdm(data, desc="Scanning for statuses"):
        for turn in battle.get('battle_timeline', []):
            p1_state = turn.get('p1_move_details')
            if p1_state and 'name' in p1_state:
                all_statuses.add(p1_state['name'])
            p2_state = turn.get('p2_move_details')
            if p2_state and 'name' in p2_state:
                all_statuses.add(p2_state['name'])
    print(all_statuses)
    return all_statuses

### 2. Basic Feature Engineering

A successful model will likely require creating many complex features. For this starter notebook, however, we will create a very simple feature set based **only on the initial team stats**. This will be enough to train a model and generate a submission file.

It's up to you to engineer more powerful features!

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def create_features(data: list[dict]) -> pd.DataFrame:
    """
    Extracts features based on the battle timeline.
    Features currently include:
    - Mean remaining HP percentage for Player 1's team.
    - Mean remaining HP percentage for Player 2's team.
    - Number of surviving Pokemon (HP > 0) for Player 1.
    - Number of surviving Pokemon (HP > 0) for Player 2.
    """
    feature_list = []

    # Creating a dictionary of pokemons along with stats in the dataset
    pokemon_dict = create_pokemon_stats_dict(data)

    # For each battle
    for battle in tqdm(data, desc="Extracting features"):
        features = {}
        timeline = battle.get('battle_timeline', [])

        # Track the conditions of teams at the end of the timeline
        # Track the number of changes of each trainer
        # Track the number of effects
        p1_n_changes, p1_effects, p1_pok_cond, p2_n_changes, p2_effects, p2_pok_cond = track_pokemon_conditions(battle)

        # Add to the features the average effectiveness
        # avg_effectiveness = avg_effectiveness_moves(battle)
        # features['avg_effectiveness'] = avg_effectiveness

        # Add to the features the mean of the percentage of HP for each team
        p1_mean_pc_hp = np.mean([info['hp'] for info in p1_pok_cond.values()])
        p2_mean_pc_hp = np.mean([info['hp'] for info in p2_pok_cond.values()])
        features['p1_mean_pc_hp'] = p1_mean_pc_hp
        features['p2_mean_pc_hp'] = p2_mean_pc_hp

        # Add to the features the number of surviving pokemon for each team
        p1_surviving_pokemon = sum(1 for info in p1_pok_cond.values() if info["hp"] > 0)
        p2_surviving_pokemon = sum(1 for info in p2_pok_cond.values() if info["hp"] > 0)
        features['p1_surviving_pokemon'] = p1_surviving_pokemon
        features['p2_surviving_pokemon'] = p2_surviving_pokemon

        # Add to the features the number of pokemon affected by status and an effect index for each team
        p1_status_score = sum(1 for i in p1_pok_cond.values() if i['hp'] > 0 and i['status'] != 'nostatus')+p1_effects
        p2_status_score = sum(1 for i in p2_pok_cond.values() if i['hp'] > 0 and i['status'] != 'nostatus')+p2_effects
        features['p1_status_score'] = p1_status_score
        features['p2_status_score'] = p2_status_score

        # Add to the features not the mean but simply the difference
        # Also some of them could not be chosen because there is redundance in the pattern of stats distribution
        speed, defense, attack, sp_attack, sp_defense, hp = compute_differences_base_stats(p1_pok_cond, p2_pok_cond, pokemon_dict)
        features['total_speed_difference'] = speed #47
        features['total_attack_difference'] = attack # 40
        features['total_defense_difference'] = defense # 19
        features['total_sp_attack_difference'] = sp_attack # 24
        features['total_sp_defense_difference'] = sp_defense # 24
        features['total_hp_difference'] = hp # 24

        # Add to the features the number of pokemon changes along the timeline for each team (indicator of strategy)
        #features['p1_n_changes'] = p1_n_changes
        #features['p2_n_changes'] = p2_n_changes

        # Add to the features the battle id and the true outcome of the battle
        features['battle_id'] = battle.get('battle_id')
        # Include target variable if in data
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])

        # Append all features to the list
        feature_list.append(features)

    # Convert to DataFrame and handle missing values introduced by get()
    return pd.DataFrame(feature_list).fillna(0)

# Create feature DataFrames for both training and test sets
print("Processing training data...")
train_df = create_features(train_data)

print("\nProcessing test data...")
test_data = []
with open(test_file_path, 'r') as f:
    for line in f:
        test_data.append(json.loads(line))
test_df = create_features(test_data)

print("\nTraining features preview:")
display(train_df.head())

### 3. Training a Baseline Model

Now that we have some features, let's train a simple `LogisticRegression` model. This will give us a starting point for our predictions.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

# Define our features (X) and target (y)
features = [col for col in train_df.columns if col not in ['battle_id', 'player_won']]
X = train_df[features]
y = train_df['player_won']

# Define the test set
X_test = test_df[features].copy()

# Set up cross validation
# [Lines to uncomment K validation]
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

val_scores1 = []
val_scores3 = []
train_scores1 = []
train_scores3 = []
test_predictions_list1 = []
test_predictions_list3 = []
print(f"Starting of {N_SPLITS}-fold cross-validation...")
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print(f"\n--- Fold {fold+1}/{N_SPLITS} ---")

    # Splitting for this fold
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Normalization
    scaler = StandardScaler()
    X_train_fold = pd.DataFrame(scaler.fit_transform(X_train_fold), columns=features, index=X_train_fold.index)
    X_val_fold = pd.DataFrame(scaler.transform(X_val_fold), columns=features, index=X_val_fold.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=features, index=X_test.index)

    # Training
    # ATTENTION for having model 2 instead of model 3, uncomment n_changes features in the feature section
    model1 = LogisticRegression(
        C=0.11,
        penalty='l2',
        solver='liblinear',
        random_state=42,
        max_iter=1000,
        class_weight=None
    )
    model3 = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=200,random_state=42)
    model1.fit(X_train_fold, y_train_fold)
    model3.fit(X_train_fold, y_train_fold)
    print(f"End of #{fold+1} training.")

    # Predict on training data to get training AUC
    train_proba1 = model1.predict_proba(X_train_fold)[:, 1]
    fold_train_auc1 = roc_auc_score(y_train_fold, train_proba1)
    train_proba3 = model3.predict_proba(X_train_fold)[:, 1]
    fold_train_auc3 = roc_auc_score(y_train_fold, train_proba3)

    # Validation
    val_preds1 = model1.predict(X_val_fold)
    val_proba1 = model1.predict_proba(X_val_fold)[:, 1]
    val_preds3 = model3.predict(X_val_fold)
    val_proba3 = model3.predict_proba(X_val_fold)[:, 1]

    fold_accuracy1 = accuracy_score(y_val_fold, val_preds1)
    fold_accuracy3 = accuracy_score(y_val_fold, val_preds3)
    fold_auc1 = roc_auc_score(y_val_fold, val_proba1)
    fold_auc3 = roc_auc_score(y_val_fold, val_proba3)
    fold_precision1 = precision_score(y_val_fold, val_preds1)
    fold_precision3 = precision_score(y_val_fold, val_preds3)
    fold_recall1 = recall_score(y_val_fold, val_preds1)
    fold_recall3 = recall_score(y_val_fold, val_preds3)
    fold_f1_1 = f1_score(y_val_fold, val_preds1)
    fold_f1_3 = f1_score(y_val_fold, val_preds3)
    val_scores1.append({'accuracy': fold_accuracy1, 'auc': fold_auc1, 'precision': fold_precision1, 'recall': fold_recall1, 'f1': fold_f1_1})
    val_scores3.append({'accuracy': fold_accuracy3, 'auc': fold_auc3, 'precision': fold_precision3, 'recall': fold_recall3, 'f1': fold_f1_3})

    train_scores1.append({'auc': fold_train_auc1})
    train_scores3.append({'auc': fold_train_auc3})
    print(f"Fold {fold+1} Accuracy Model 1: {fold_accuracy1:.4f}, Train AUC: {fold_train_auc1:.4f}, Val AUC: {fold_auc1:.4f}, Precision: {fold_precision1:.4f}, Recall: {fold_recall1:.4f}, F1: {fold_f1_1:.4f}")
    print(f"Fold {fold+1} Accuracy Model 3: {fold_accuracy3:.4f}, Train AUC: {fold_train_auc3:.4f}, Val AUC: {fold_auc3:.4f}, Precision: {fold_precision3:.4f}, Recall: {fold_recall3:.4f}, F1: {fold_f1_3:.4f}")

    # Generating predictions
    fold_test_preds1 = model1.predict_proba(X_test_scaled)[:, 1]
    test_predictions_list1.append(fold_test_preds1)
    fold_test_preds3 = model3.predict_proba(X_test_scaled)[:, 1]
    test_predictions_list3.append(fold_test_preds3)

# Printing of metrics
print("\n--- Cross-validation completed Model 1---")
mean_train_auc1 = np.mean([s['auc'] for s in train_scores1])
mean_train_auc3 = np.mean([s['auc'] for s in train_scores3])
mean_accuracy1 = np.mean([s['accuracy'] for s in val_scores1])
mean_accuracy3 = np.mean([s['accuracy'] for s in val_scores3])
mean_auc1 = np.mean([s['auc'] for s in val_scores1])
mean_auc3 = np.mean([s['auc'] for s in val_scores3])
mean_precision1 = np.mean([s['precision'] for s in val_scores1])
mean_precision3 = np.mean([s['precision'] for s in val_scores3])
mean_recall1 = np.mean([s['recall'] for s in val_scores1])
mean_recall3 = np.mean([s['recall'] for s in val_scores3])
mean_f1_1 = np.mean([s['f1'] for s in val_scores1])
mean_f1_3 = np.mean([s['f1'] for s in val_scores3])
print(f"Average Training AUC: {mean_train_auc1:.4f}")
print(f"Average Validation Accuracy: {mean_accuracy1:.4f}")
print(f"Average Validation AUC: {mean_auc1:.4f}")
print(f"Average Validation Precision: {mean_precision1:.4f}")
print(f"Average Validation Recall: {mean_recall1:.4f}")
print(f"Average Validation F1: {mean_f1_1:.4f}")
print("\n--- Cross-validation completed Model 3---")
print(f"Average Training AUC: {mean_train_auc3:.4f}")
print(f"Average Validation Accuracy: {mean_accuracy3:.4f}")
print(f"Average Validation AUC: {mean_auc3:.4f}")
print(f"Average Validation Precision: {mean_precision3:.4f}")
print(f"Average Validation Recall: {mean_recall3:.4f}")
print(f"Average Validation F1: {mean_f1_3:.4f}")
# [Lines to uncomment K validation]

### 4. Creating the Submission File

The competition requires a `.csv` file with two columns: `battle_id` and `player_won`. Let's use our trained model to make predictions on the test set and format them correctly.

In [ ]:
print("Generation of final predictions over the test set...")

# [Lines to uncomment K validation]
# Compute the average of the k models
average_test_proba1 = np.mean(test_predictions_list1, axis=0)
average_test_proba3 = np.mean(test_predictions_list3, axis=0)

# combined_average_proba = (average_test_proba1 + average_test_proba3) / 2.0
# final_test_predictions = (combined_average_proba > 0.5).astype(int)

# Convert to binary predictions
final_test_predictions1 = (average_test_proba1 > 0.5).astype(int)
final_test_predictions3 = (average_test_proba3 > 0.5).astype(int)
# [Lines to uncomment K validation]

# [Lines to comment K validation]
# final_test_predictions = (test_predictions > 0.5).astype(int)
# [Lines to comment K validation]

# Create DataFrame for submission
submission_df1 = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': final_test_predictions1
})

submission_df3 = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': final_test_predictions3
})

# Save the csv file
submission_df1.to_csv('submission1.csv', index=False)
submission_df3.to_csv('submission3.csv', index=False)


print("\n'submission.csv' file successfully created!")
display(submission_df1.head())
display(submission_df3.head())

### 5. Submitting Your Results

Once you have generated your `submission.csv` file, there are two primary ways to submit it to the competition.

---

#### Method A: Submitting Directly from the Notebook

This is the standard method for code competitions. It ensures that your submission is linked to the code that produced it, which is crucial for reproducibility.

1.  **Save Your Work:** Click the **"Save Version"** button in the top-right corner of the notebook editor.
2.  **Run the Notebook:** In the pop-up window, select **"Save & Run All (Commit)"** and then click the **"Save"** button. This will run your entire notebook from top to bottom and save the output, including your `submission.csv` file.
3.  **Go to the Viewer:** Once the save process is complete, navigate to the notebook viewer page.
4.  **Submit to Competition:** In the viewer, find the **"Submit to Competition"** section. This is usually located in the header of the output section or in the vertical "..." menu on the right side of the page. Clicking the **Submit** button this will submit your generated `submission.csv` file.

After submitting, you will see your score in the **"Submit to Competition"** section or in the [Public Leaderboard](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?).

---

#### Method B: Manual Upload

You can also generate your predictions and submission file using any environment you prefer (this notebook, Google Colab, or your local machine).

1.  **Generate the `submission.csv` file** using your model.
2.  **Download the file** to your computer.
3.  **Navigate to the [Leaderboard Page](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?)** and click on the **"Submit Predictions"** button.
4.  **Upload Your File:** Drag and drop or select your `submission.csv` file to upload it.

This method is quick, but keep in mind that for the final evaluation, you might be required to provide the code that generated your submission.

Good luck!